In [ ]:
#ollama run brxce/monadgpt modern french
#ollama

In [1]:
!ls data/

liv0232_ideedelaperfectiondelape.txt


# Normalisation
`!git clone https://github.com/rbawden/ModFr-Norm.git`

In [ ]:
!pip install torch transformers datasets

In [27]:
import re

def regroup_lines_into_sentences(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            sentences = []
            current_sentence = ""

            for line in lines:
                line = line.strip()  # Supprimer les espaces en début et fin de ligne
                if line:  # Vérifier si la ligne n'est pas vide
                    # Ajouter la ligne à la phrase courante
                    current_sentence += line + " "
                    if re.search(r'[.!?]$', line):  # Vérifier si la ligne se termine par un marqueur de fin de phrase
                        sentences.append(current_sentence.strip())  # Ajouter la phrase complète à la liste
                        current_sentence = ""  # Réinitialiser la phrase courante
                    current_sentence = current_sentence.replace('¬ ', '')  # Remplacer '¬ ' par rien
            return sentences
    except FileNotFoundError:
        return "Fichier non trouvé."

file_path = "data/liv0232_ideedelaperfectiondelape.txt"  # Remplacez ceci par le chemin de votre fichier .txt
sentences = regroup_lines_into_sentences(file_path)

In [ ]:
from transformers import pipeline
normaliser = pipeline(model="rbawden/modern_french_normalisation", batch_size=32, beam_size=5, cache_file="./cache.pickle", trust_remote_code=True)

with open("normalized.txt", "w", encoding="utf-8") as output_file:
  for line in sentences:
    normalized_line = normaliser([line])
    output_file.write(normalized_line[0]['text'].strip() + "\n")
    print(line)
    print(normalized_line[0]['text'].strip() + "\n")

# Analyse document
https://github.com/langchain-ai/langchain/blob/master/cookbook/analyze_document.ipynb

https://python.langchain.com/docs/use_cases/summarization/

In [ ]:
!pip install langchain

In [28]:
with open("data/normalized.txt") as f:
    state_of_the_union = f.read()

In [29]:
from langchain.chains import AnalyzeDocumentChain
from langchain_community.llms import Ollama

llm = Ollama(model="mixtral:8x7b")

In [30]:
from langchain.chains.question_answering import load_qa_chain

qa_chain = load_qa_chain(llm, chain_type="map_reduce")

In [31]:
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

In [32]:
qa_document_chain.run(
    input_document=state_of_the_union,
    question="What does the author think about estampes?",
)

" The author does not express a specific opinion about estampes in general, but they do mention that some prints or academic studies must be adjusted according to perspective and the original viewpoint. They also praise a particular engraving for its merit, grandeur, rare considerations, and admirable expression. Additionally, they criticize another engraving for lacking force in expressing violent passions. However, these comments are specific to individual works rather than estampes as a whole. Therefore, it is not possible to determine the author's overall opinion about estampes based on this document alone."

In [33]:
qa_document_chain.run(
    input_document=state_of_the_union,
    question="Summarizes the document and the author's thinking",
)

' The document is a book written by Roland Freart Sieur de Chambray titled "IDE DE DE PERFECTION DE LA PEINTURE" and published in Le Mans by IA CQVESYs AMBART. The book aims to demonstrate the principles of art and painting through examples, comparing ancient painters like Leonardo da Vinci, Raphael, Julius Roman, and Poussin with some modern French painters. The author emphasizes the importance of perspective in painting and provides axioms for it. The document also highlights the significance of accurately depicting Costumes to enhance the quality of a painter\'s work and avoid criticism.'

In [34]:
qa_document_chain.run(
    input_document=state_of_the_union,
    question="Give me a list of all the individuals cited by the author in these arguments",
)

' The president did not mention Michael Jackson.\n\nThe individuals cited by the author in these arguments are:\n\n1. Géomettres (geometers)\n2. Peintres (painters)\n3. Es (artists or people who make engravings)\n4. Madame (a person to whom the document is addressed)\n5. Raphael\n6. Marc-Antoine\n7. The women in the engraving\n8. The bourreaux (executioners) in the engraving\n9. The assassins of "ces urs petits In-ô"\n10. The malheureuses mères (unhappy mothers) of the children being killed\n11. The Peintre (the Painter), who is Raphael.\n12. NÔTRE SEIGNEUR (Jesus Christ)\n13. La Vierge (Virgin Mary)\n14. Les Maries (the Marys, plural form of Marie in French, likely referring to Mary Magdalene and other female followers of Jesus)\n15. Joseph JRimathie (Joseph of Arimathea)\n16. Nicodeme (Nicodemus)\n17. Saint Jean (John the Apostle)\n18. Ioseph d\'Arimathie (Joseph of Arimathea, mentioned again)\n19. La PEINTVRE (the Painter, generic term, not an individual)\n20. The author himself\n21

In [ ]:
# citation -> https://python.langchain.com/docs/use_cases/question_answering/citations/

# RAG 
https://github.com/langchain-ai/langchain/blob/master/cookbook/Semi_structured_multi_modal_RAG_LLaMA2.ipynb